### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__FAO Fisheries and Aquaculture__  
__Global Aquaculture Production__  
  URL:  
    ・https://www.fao.org/fishery/en/collection/aquaculture  
  DATA:  
    　取得したZIPファイルを解凍後、`Aquaculture_2021.1.2`フォルダをそのまま`data`ディレクトリに配置します。  
    ・Aquaculture_2021.1.2/AQUACULTURE_QUANTITY.csv  
    ・Aquaculture_2021.1.2/CL_FI_COUNTRY_GROUPS.csv  
    ・Aquaculture_2021.1.2/CL_FI_SPECIES_GROUPS.csv  
  データの取得方法:  
    「Aquaculture Production (CSV raw data)」のリンクをクリックします。

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

### 1. データの概要を掴む

3つのデータをファイルからロードします。

養殖業生産量データから始めます。

In [ ]:
aquaculture_quantity_df = pd.read_csv('data/Aquaculture_2021.1.2/AQUACULTURE_QUANTITY.csv')

print(f"{len(aquaculture_quantity_df):,}件")

中身を確認。

In [ ]:
aquaculture_quantity_df.head()

`info()`で各列の状況を確認。

不明な列はありますか？

今回のデータの定義はダウンロードしたファイル「DSD_FI_AQUACULTURE.xlsx」にすべて記載されています。  
※「DSD」とは「data set definition」の意味です。

In [ ]:
aquaculture_quantity_df.info()

国・グループのデータをロードします。

In [ ]:
country_group_df = pd.read_csv('data/Aquaculture_2021.1.2/CL_FI_COUNTRY_GROUPS.csv')

print(f"{len(country_group_df):,}件")

中身を確認。

In [ ]:
country_group_df.head()

`info()`で各列の状況を確認。

不明な列はありますか？

In [ ]:
country_group_df.info()

英語以外の言語の列は使用しないので削除します。

In [ ]:
country_group_df = country_group_df.filter(regex="^(?!.*_(Fr|Es|Ar|Cn|Ru)$)", axis=1)

country_group_df.info()

中身を確認。

In [ ]:
country_group_df.head()

魚の種類のデータをロードします。

In [ ]:
species_group_df = pd.read_csv('data/Aquaculture_2021.1.2/CL_FI_SPECIES_GROUPS.csv')

print(f"{len(species_group_df):,}件")

中身を確認。

先頭行の「Lampreys」(lampreeの複数形)、「nei」とは一体何でしょうか？

参考）  
__What does NEI stand for?__  
https://www.abbreviations.com/NEI

In [ ]:
species_group_df.head()

`info()`で各列の状況を確認。

不明な列はありますか？

In [ ]:
species_group_df.info()

英語以外の言語の列は使用しないので削除します。

In [ ]:
species_group_df = species_group_df.filter(regex="^(?!.*_(Fr|Es|Ar|Cn|Ru)$)", axis=1)

species_group_df.info()

### 2. キーを使ったデータの結合

２つのデータフレームをキーを指定して結合していきます。

生産量と国・グループのデータフレームを以下の方法で結合します。

- 両データフレームのキーをそれぞれ指定する。
- 生産量の全データを維持したいので左結合とする。  
- 結合後は国・グループ側のキーを削除する。

In [ ]:
quantity_country_df = pd.merge(aquaculture_quantity_df,
                               country_group_df,
                               left_on='COUNTRY.UN_CODE',
                               right_on='UN_Code',
                               how="left")

quantity_country_df = quantity_country_df.drop(columns='UN_Code')

quantity_country_df.head()

`info()`で各列の状況を確認。

In [ ]:
quantity_country_df.info()

`Identifier`と2つの`Name`が何の識別子と名前か分からなくなってしまっているので、各列名を先頭に`COUNTRY.`を付け加えて変更します。

In [ ]:
quantity_country_df = quantity_country_df.rename(columns={
                        'Identifier':'COUNTRY.Identifier',
                        'Name_En':'COUNTRY.Name_En',
                        'Official_Name_En':'COUNTRY.Official_Name_En'})

quantity_country_df.info()

次に魚の種類のデータフレームを以下の方法で結合します。

- 両データフレームのキーをそれぞれ指定する。
- 生産量の全データを維持したいので左結合とする。  
- 結合後は魚の種類側のキーを削除する。

In [ ]:
quantity_country_species_df = pd.merge(quantity_country_df, species_group_df,
                                       left_on='SPECIES.ALPHA_3_CODE', right_on='3A_Code', how="left")

quantity_country_species_df = quantity_country_species_df.drop(columns='3A_Code')

quantity_country_species_df.head()

`info()`で各列の状況を確認。

In [ ]:
quantity_country_species_df.info()

先ほどと同じように、追加された`Identifier`と2つの`Name`の先頭に`SPECIES.`を付け加えて変更します。

In [ ]:
quantity_country_species_df = quantity_country_species_df.rename(columns={
                                'Identifier':'SPECIES.Identifier',
                                'Name_En':'SPECIES.Name_En',
                                'Scientific_Name':'SPECIES.Scientific_Name'})

quantity_country_species_df.info()

これでやっと分析できる状態となりました。  
手始めに世界の養殖業の生産量の推移を棒グラフで表示してみましょう。

まずは年ごとにグループ化し、生産量の合計を計算したデータフレームを作成します。

In [ ]:
world_quantity_groupby_year = quantity_country_species_df.groupby('PERIOD')['VALUE'].sum()

world_quantity_groupby_year = world_quantity_groupby_year.reset_index()

world_quantity_groupby_year

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
sns.set(style='white', font=['MS Gothic', 'Hiragino Sans', 'TakaoPGothic']) # 日本語フォントは設定が必要

plt.figure(figsize=(20,10))
ax = sns.barplot(x='PERIOD', y='VALUE', data=world_quantity_groupby_year)

ax.set_xlabel("年")
ax.xaxis.set_major_locator(ticker.MultipleLocator(5)) # 5年刻み

ax.set_ylabel("生産量（万トン）");
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/10_000))) # 万単位

### 3. データフレームを縦に連結する

今度は国別の養殖業の生産量の推移を積み上げ折れ線グラフで表示してみましょう。  
最近年での上位の国とそれ以外の国を1つにまとめたものを対象とします。

まず最新年を取得します。

In [ ]:
year_max = quantity_country_species_df['PERIOD'].max()

year_max

まず上位20位の国を取得してみます。

In [ ]:
top20_countries_in_2019_df = quantity_country_species_df[quantity_country_species_df['PERIOD'] == year_max] \
                            .groupby('COUNTRY.Name_En')['VALUE'].sum() \
                            .reset_index() \
                            .sort_values('VALUE', ascending=False)[:20]

top20_countries_in_2019_df

日本が13位にいるのと、14位から数字が大きく下がるので、今回は上位13位までを対象にします。

上位13位の国名のリストを取得します。

In [ ]:
top13_countries = top20_countries_in_2019_df[:13]['COUNTRY.Name_En'].tolist()

top13_countries

生産量を13ヶ国に絞り、国と年でグループ化して各年の生産量を計算した`Series`オブジェクトを作成します。

In [ ]:
top13_quantity_groupby_country_year_s = quantity_country_species_df[ \
                              quantity_country_species_df['COUNTRY.Name_En'].isin(top13_countries)] \
                              .groupby(['COUNTRY.Name_En', 'PERIOD'])['VALUE'] \
                              .sum()

top13_quantity_groupby_country_year_s

次に`その他`の年ごとの生産量を計算した`Series`オブジェクトを作成します。

世界生産量から13ヶ国合計の生産量を引いて計算できますので、先に年ごとの世界生産量を格納した`Series`オブジェクトを作成します。

In [ ]:
world_quantity_groupby_year_s = world_quantity_groupby_year.set_index('PERIOD')['VALUE']

world_quantity_groupby_year_s

次に、年ごとの13ヶ国合計の生産量を格納した`Series`オブジェクトを作成します。

In [ ]:
top13_quantity_groupby_year_s = top13_quantity_groupby_country_year_s.groupby('PERIOD').sum()

top13_quantity_groupby_year_s

作成した年ごとの世界生産量を格納した`Series`から、13ヶ国の年ごとの生産量を格納した`Series`を引きます。

In [ ]:
the_others_quantity_groupby_year_s = world_quantity_groupby_year_s - top13_quantity_groupby_year_s

the_others_quantity_groupby_year_s

作成した`Series`オブジェクトを13ヶ国の`Series`オブジェクトを結合しますので、同じ階層型インデックス(multiindex)を持つように変形します。

インデックスを一旦リセットしてから国名の列を追加し、改めて２つの列をインデックスにしましょう。

国名は`その他`にします。

In [ ]:
the_others_quantity_groupby_year_s = the_others_quantity_groupby_year_s.reset_index()

the_others_quantity_groupby_year_s['COUNTRY.Name_En'] = 'その他'

the_others_quantity_groupby_year_s = the_others_quantity_groupby_year_s \
                                    .set_index(['COUNTRY.Name_En',  'PERIOD'])['VALUE']

the_others_quantity_groupby_year_s

13ヶ国のSeriesオブジェクトと`その他`のSeriesオブジェクトを`concat()`で縦に連結します。

In [ ]:
top13_the_others_quantity_s = pd.concat(
                                [top13_quantity_groupby_country_year_s, the_others_quantity_groupby_year_s])

top13_the_others_quantity_s

最後に、年に抜けがないか確認しましょう。  
データに不備があったり、1950年以降に誕生した国がある場合はデータ数が異なってしまいます。

生成した`Series`オブジェクトを国ごとにグループ化し、各国の年の件数、最大値、最小値を表示してみます。

In [ ]:
top13_the_others_quantity_s.reset_index('PERIOD') \
                           .groupby('COUNTRY.Name_En')['PERIOD'] \
                           .agg(['size', 'min', 'max'])

問題なさそうなので、Seriesオブジェクトのインデックスを外してデータフレームにします。

これで積み上げ折れ線グラフに必要なデータが揃いました。

In [ ]:
top13_the_others_quantity_df = top13_the_others_quantity_s.reset_index()

top13_the_others_quantity_df

`seaborn`には積み上げ折れ線グラフはありませんので、`matplotlib`の`stackplot()`を利用します。

データフレームを変形して、stackplotの引数として必要な以下の３つのリストを用意する必要があります。

1. 年のリスト
2. （国の数）X（年の数）の二次元の生産量のリスト
3. ラベル用の国名のリスト

参考）  
__matplotlib.pyplot.stackplot__  
https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.stackplot.html

まずは年のリストを取得します。

In [ ]:
years = top13_the_others_quantity_df['PERIOD'].unique()

print(f"年の件数: {len(years)}件")

years

次は二次元の生産量のリストを取得したいところなのですが、データフレームを見ると生産量の多い順であった国の順番が狂ってしまっています。  
先にラベル用の国名のリストを生成し、それに合わせて順序を整えるようにします。

先ほど作成した上位13ヶ国の国名のリストに`その他`を加え、ラベル用の国名リストを生成します。

In [ ]:
top13_countries_and_the_others = top13_countries + ["その他"]

top13_countries_and_the_others

データフレームを国名でソートできるよう`COUNTRY.Name_En`のデータ型を`category`とし、カテゴリとして先ほどの国名リストを設定します。

In [ ]:
top13_the_others_quantity_df['COUNTRY.Name_En']= \
    top13_the_others_quantity_df['COUNTRY.Name_En'].astype('category')

top13_the_others_quantity_df['COUNTRY.Name_En'] = \
    top13_the_others_quantity_df['COUNTRY.Name_En'].cat.reorder_categories(top13_countries_and_the_others)

top13_the_others_quantity_df['COUNTRY.Name_En']

ソートする準備が調ったので、データフレームを国名、年でソートします。

In [ ]:
top13_the_others_quantity_df = top13_the_others_quantity_df.sort_values(['COUNTRY.Name_En', 'PERIOD'])

top13_the_others_quantity_df

それでは二次元の生産量のリストを取得します。

国でグループ化してからグループごとに`apply()`で生産量をリスト化し、最後にそれらを１つのリストにします。

In [ ]:
values = top13_the_others_quantity_df.groupby('COUNTRY.Name_En')['VALUE'].apply(list).tolist()

print(f"各次元の要素数: {np.shape(values)}")

# 先頭2行だけ表示
values[:2]

ではグラフを表示しましょう。

どんなことが読み取れますか？

In [ ]:
sns.set_palette('pastel')

fig, ax = plt.subplots(figsize=(20,10))
plt.stackplot(years, values, labels=top13_countries_and_the_others)

ax.set_xlabel("年")

ax.set_ylabel("生産量（万トン）");
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/10_000))) # 万単位

handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc="upper left", bbox_to_anchor=(1.012, 1.014)) # ラベルの順序を逆さにする

### 4. 過去・未来のデータとの結合

ここでは増加率と移動平均を取得する目的で、過去および未来のデータとの結合をおこないます。

#### 4-1. 過去の1件前のデータを取得

先ほどの国別の養殖業の生産量のグラフでは中国以外の増減がやや読みにくいので、各国の増加率の折れ線グラフを表示します。

グラフ用のデータフレームの作成手順は以下のようになります。
1. 先ほどのグラフ用のデータフレームに各国の各年ごとに前年の生産量の値を新たな列`VALUE_previous_year`に格納する。
2. 前年・当年の生産量から増加率を計算し、新たな列`increase_rate`に格納する。

まずデータを年度の昇順にソートします。

普通にソートしても良いのですが、必要最低限のソートになるよう、グループ単位でソートします。

In [ ]:
top13_the_others_quantity_df_sorted = top13_the_others_quantity_df \
                                       .groupby('COUNTRY.Name_En') \
                                       .apply(lambda g: g.sort_values('PERIOD'))

top13_the_others_quantity_df_sorted

次に`groupby()`で魚種でグループ化した`DataFrameGroupBy`オブジェクトを作成後、`shift()`を実行します。  

前年の生産量を求めたいので、`shift()`の引数に`periods=1`を指定します。

In [ ]:
top13_the_others_previous_year_s = top13_the_others_quantity_df_sorted \
                          .groupby('COUNTRY.Name_En')['VALUE'] \
                          .shift(periods=1)

top13_the_others_previous_year_s

作成した`Series`を元のデータフレームに`VALUE_previous_year`列として加えます。

In [ ]:
top13_the_others_quantity_df['VALUE_previous_year'] = top13_the_others_previous_year_s

top13_the_others_quantity_df

さらに、前年・当年の生産量から増加率を計算し、新たな列`increase_rate`に格納します。

増加率は小数点第一で丸めます。

In [ ]:
top13_the_others_quantity_df['increase_rate'] = round( \
 (top13_the_others_quantity_df['VALUE'] / top13_the_others_quantity_df['VALUE_previous_year'] - 1) * 100)

top13_the_others_quantity_df

念のため、ミャンマーのデータで移動平均が正しく計算されているか確認します。

In [ ]:
top13_the_others_quantity_df.query('`COUNTRY.Name_En` == "Myanmar"') # 「.」が含まれているのでバッククォートで囲む

計算は合っていそうですが、`inf`があります。  
何が原因でしょうか？

`inf`がいくつあるのか確認します。

In [ ]:
np.isinf(top13_the_others_quantity_df['increase_rate']).sum()

`inf`を`NaN`に置き換えてしまってもいいのですが、そもそもの計算の仕方が悪かったので、もう一度やり直します。

In [ ]:
top13_the_others_quantity_df['increase_rate'] = top13_the_others_quantity_df.apply( \
    lambda x: round((x['VALUE'] / x['VALUE_previous_year'] - 1) * 100, 1) \
                if x['VALUE_previous_year'] > 0 else np.NaN, axis=1)

top13_the_others_quantity_df

もう一度`inf`がいくつあるのか確認します。

In [ ]:
np.isinf(top13_the_others_quantity_df['increase_rate']).sum()

それでは増加率の折れ線グラフを表示します。

どんなことが読み取れますか？

In [ ]:
top13_the_others_quantity_df[top13_the_others_quantity_df['COUNTRY.Name_En'] == 'China'].head(50)

In [ ]:
### チャレンジしてみましょう！！ ###

かなり突出した数値が幾つかあります。  
データを確認してみます。

In [ ]:
top13_the_others_quantity_df[top13_the_others_quantity_df['increase_rate'] > 100]

突出した数値となった原因について、簡単に調査してみましょう。

In [ ]:
### チャレンジしてみましょう！！ ###

ここからは魚種別の生産量の推移を分析していきます。

魚種の分類方法がいくつかあるようですが、今回はISSCAAP（水棲動植物国際標準統計分類）で分類します。

まずは上位20位の魚種を取得します。

In [ ]:
top20_species_s = quantity_country_species_df[quantity_country_species_df['PERIOD'] == year_max] \
                  .groupby('ISSCAAP_Group_En')['VALUE'].sum().sort_values(ascending=False)[:20]

top20_species_s

11位以降は数字が大きく下がることもあり、今回は上位10位までを対象とします。  
ただし「Miscellaneous freshwater fishes」「Freshwater crustaceans」は特定の種ではないので、この２つを除いた8種を対象とします。

上位8位の国名のリストを取得します。

In [ ]:
top10_species_s = top20_species_s[:10]

top8_species_s = top10_species_s.loc[
    ~top10_species_s.index.isin(['Miscellaneous freshwater fishes', 'Freshwater crustaceans'])]

top8_species = top8_species_s.index.tolist()

top8_species

生産量を8種に絞り、種類と年でグループ化して各年の生産量を計算した`Series`オブジェクトを作成します。

In [ ]:
top8_quantity_groupby_year_s = quantity_country_species_df[ \
                                quantity_country_species_df['ISSCAAP_Group_En'].isin(top8_species)] \
                                .groupby(['ISSCAAP_Group_En', 'PERIOD'])['VALUE'] \
                                .sum()

top8_quantity_groupby_year_s

年に抜けがないか確認しましょう。

生成した`Series`オブジェクトを国ごとにグループ化し、各国の年の件数、最大値、最小値を表示してみます。

In [ ]:
top8_quantity_groupby_year_s.reset_index('PERIOD') \
                            .groupby('ISSCAAP_Group_En')['PERIOD'] \
                            .agg(['size', 'min', 'max'])

グラフ表示の関数の引数に合わせるため、インデックスをすべてリセットします。

In [ ]:
top8_quantity_df = top8_quantity_groupby_year_s.reset_index()

top8_quantity_df

英語の魚名はあまり馴染みなく分かりにくいので、対応する日本語と中国語の訳を用意しました。   
英語・中国語・日本語のデータフレームを作成し、それを8種の生産量データフレームに結合します。

|SPECIES.Name_En|SPECIES.Name_Cn|SPECIES.Name_Jp|
|:---|:---|:---|
|Carps, barbels and other cyprinids|鲤鱼・鲫鱼|コイ・フナ類|
|Red seaweeds|红藻|紅藻類（ノリ等）|
|Brown seaweeds|褐藻|褐藻類（コンブ等）|
|Shrimps, prawns|虾|エビ類|
|Tilapias and other cichlids|罗非鱼|ティラピア類|
|Oysters|牡蛎|カキ類|
|Clams, cockles, arkshells|蛤仔・蛤蜊|アサリ・ハマグリ類|
|Salmons, trouts, smelts|鲑鱼・鳟鱼|サケ・マス類|

まず英語・中国語・日本語のデータフレームを作成します。

In [ ]:
isscaap_group_df = pd.DataFrame([
        ['Carps, barbels and other cyprinids', '鲤鱼・鲫鱼', 'コイ・フナ類'],
        ['Red seaweeds', '红藻', '紅藻類（ノリ等）'],
        ['Brown seaweeds', '褐藻', '褐藻類（コンブ等）'],
        ['Shrimps, prawns', '虾', 'エビ類'],
        ['Tilapias and other cichlids', '罗非鱼', 'ティラピア類'],
        ['Oysters', '牡蛎', 'カキ類'],
        ['Clams, cockles, arkshells', '蛤仔・蛤蜊', 'アサリ・ハマグリ類'],
        ['Salmons, trouts, smelts', '鲑鱼・鳟鱼', 'サケ・マス類']],
    columns=['ISSCAAP_Group_En', 'ISSCAAP_Group_Cn', 'ISSCAAP_Group_Jp'])

isscaap_group_df

作成したデータフレームを8種の生産量データフレームに結合します。

In [ ]:
top8_quantity_df = pd.merge(top8_quantity_df, isscaap_group_df, on='ISSCAAP_Group_En', how='left')

top8_quantity_df

最後に魚種でソートできるよう、`ISSCAAP_Group_En`のデータ型を`category`とし、カテゴリとして先ほどの魚種のリストを設定します。

In [ ]:
top8_quantity_df['ISSCAAP_Group_En'] = top8_quantity_df['ISSCAAP_Group_En'].astype('category')

top8_quantity_df['ISSCAAP_Group_En'] = top8_quantity_df['ISSCAAP_Group_En'].cat.reorder_categories(top8_species)

top8_quantity_df['ISSCAAP_Group_En']

ソートする準備が調ったので、データフレームを魚種、年でソートします。

In [ ]:
top8_quantity_df = top8_quantity_df.sort_values(['ISSCAAP_Group_En', 'PERIOD'])

top8_quantity_df

グラフを表示します。

どんなことが読み取れますか？

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))

sns.lineplot(x='PERIOD', y='VALUE', hue='ISSCAAP_Group_En', data=top8_quantity_df)

ax.set_xlabel("年")

ax.set_ylabel("生産量（万トン）");
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/10_000))) # 万単位

# 言語混合ラベル
# 中国語を表示するにはOSにフォントのインストールが必要
labels = top8_quantity_df['ISSCAAP_Group_En'].cat.categories + ' / ' \
          + top8_quantity_df['ISSCAAP_Group_Jp'].unique()
ax.legend(labels, loc="upper left", bbox_to_anchor=(1.01, 1.01))

#### 4-2. 移動平均の取得

8魚種のうち上位以外の魚種は線の上下が激しく交差も多いので、見やすいように5年間の単純移動平均を求めてグラフに表示してみます。

グラフ表示に必要なデータフレームの作成手順は以下のようになります。
1. 魚種単位で年度の昇順にソートする。
2. `rolling()`を使って各行ごとに自身と前後の行の平均、つまり5年間の単純移動平均を計算したデータフレームを作成。
3. 元のデータフレームに単純移動平均のデータフレームを結合。

まずデータを年度の昇順にソートします。

先ほどと同じように、必要最低限のソートになるよう、グループ単位でソートします。

In [ ]:
top8_quantity_df_sorted = top8_quantity_df.groupby('ISSCAAP_Group_En').apply(lambda g: g.sort_values('PERIOD'))

top8_quantity_df_sorted

次に`groupby()`で魚種でグループ化した`DataFrameGroupBy`オブジェクトを作成後、`rolling()`を実行します。  

自身の行を真ん中として前後2行ずつの3行の平均を求めたいので、`rolling()`の引数に`window=5`、`center=True`を指定し、`mean()`を実行します。

In [ ]:
top8_moving_average_s = top8_quantity_df_sorted \
                          .groupby('ISSCAAP_Group_En')['VALUE'] \
                          .rolling(window=5, center=True) \
                          .mean()

top8_moving_average_s

作成した5年移動平均のデータフレームを元のデータフレームと結合したいので、２つのインデックスのうち`ISSCAAP_Group_En`の方を削除します。

In [ ]:
top8_moving_average_s.reset_index(level='ISSCAAP_Group_En', drop=True, inplace=True)

top8_moving_average_s

作成した`Series`を元のデータフレームに`VALUE_移動平均_5年`列として加えます。

In [ ]:
top8_quantity_df_sorted['VALUE_移動平均_5年'] = top8_moving_average_s

top8_quantity_df_sorted

念のため、`サケ・マス類`のデータを年でソートし、移動平均が正しく計算されているか確認します。

In [ ]:
top8_quantity_df_sorted.query('ISSCAAP_Group_En == "Salmons, trouts, smelts"').sort_values('PERIOD')

それでは8魚種の生産量の推移とその5年移動平均の2つの折れ線グラフを表示します。

どんなことが読み取れますか？

In [ ]:
sns.set_palette('Set1') # 区別しやすいように色のセットを変更

fig, axes = plt.subplots(1, 2, figsize=(20,12), sharey=True)
fig.tight_layout()

# 通常のグラフ
sns.lineplot(x='PERIOD', y='VALUE', hue='ISSCAAP_Group_En', data=top8_quantity_df, ax=axes[0])

axes[0].set_xlabel("年")

axes[0].set_ylabel("生産量（万トン）");
axes[0].yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, pos: '{:,.0f}'.format(y/10_000))) # 万単位

axes[0].legend([],[], frameon=False) # 凡例を非表示

# 移動平均グラフ
sns.lineplot(x='PERIOD', y='VALUE_移動平均_5年', hue='ISSCAAP_Group_En', data=top8_quantity_df_sorted,
             ax=axes[1])
axes[1].set_xlabel("年")

# 多言語混合ラベル
# 中国語を表示するにはOSにフォントのインストールが必要
labels = top8_quantity_df['ISSCAAP_Group_En'].cat.categories + ' / ' \
          + top8_quantity_df['ISSCAAP_Group_Jp'].unique()
axes[1].legend(labels, loc="upper left", bbox_to_anchor=(0.52, 1.2))

### 5. 水産庁の分析結果と比較

今回の分析は以下の水産庁の分析手順をなぞったものです。  
見解の違いがありますか？

水産庁 （1）世界の漁業・養殖業生産  
https://www.jfa.maff.go.jp/j/kikaku/wpaper/r02_h/trend/1/t1_4_1.html